In [2]:
import pandas as pd

gbif_vern = pd.read_csv(r"C:\Users\anormand\Documents\Projet Python\Biodiv\Data\TAXO_GBIF\backbone\VernacularName.tsv",
                     delimiter="\t")
gbif_Description = pd.read_csv(r"C:\Users\anormand\Documents\Projet Python\Biodiv\Data\TAXO_GBIF\backbone\Description.tsv",
                     delimiter="\t")
gbif_Distribution = pd.read_csv(r"C:\Users\anormand\Documents\Projet Python\Biodiv\Data\TAXO_GBIF\backbone\Distribution.tsv",
                     delimiter="\t")
gbif_Reference = pd.read_csv(r"C:\Users\anormand\Documents\Projet Python\Biodiv\Data\TAXO_GBIF\backbone\Reference.tsv",
                     delimiter="\t")

C:\Users\anormand\AppData\Local\Temp\ipykernel_20160\349408218.py:3: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  gbif_vern = pd.read_csv(r"C:\Users\anormand\Documents\Projet Python\Biodiv\Data\TAXO_GBIF\backbone\VernacularName.tsv",
C:\Users\anormand\AppData\Local\Temp\ipykernel_20160\349408218.py:7: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  gbif_Distribution = pd.read_csv(r"C:\Users\anormand\Documents\Projet Python\Biodiv\Data\TAXO_GBIF\backbone\Distribution.tsv",


In [3]:

gbif_Taxon = pd.read_csv(r"C:\Users\anormand\Documents\Projet Python\Biodiv\Data\TAXO_GBIF\backbone\Taxon.tsv",
                     delimiter="\t",
                    on_bad_lines='skip')
gbif_TypesAndSpecimen = pd.read_csv(r"C:\Users\anormand\Documents\Projet Python\Biodiv\Data\TAXO_GBIF\backbone\TypesAndSpecimen.tsv",
                     delimiter="\t")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\anormand\\Documents\\Projet Python\\Biodiv\\Data\\TAXO_GBIF\\Taxon.tsv'

In [4]:
df_biodiv= pd.read_csv(r"C:\Users\anormand\Documents\Projet Python\Biodiv\Data\GBIF_Costa_Rica\data_GBIF_Costa_Rica_codeMaille5Km_periodes.csv",
    dtype={
        'nombreObs': int,   # `nombreObs` en float
        'speciesKey': int     # `speciesKey` en entier
    })

In [5]:

gbif_vern

,taxonID,vernacularName,language,country,countryCode,sex,lifeStage,source
0,5371864,sels næp,no,NaN,NaN,NaN,NaN,Nordic plant uses from Gunnerus and Høeg
1,5371864,spreng-rood,no,NaN,NaN,NaN,NaN,Nordic plant uses from Gunnerus and Høeg
2,5371864,syle-næbber,no,NaN,NaN,NaN,NaN,Nordic plant uses from Gunnerus and Høeg
3,3034225,angelsrot,NaN,NaN,NaN,NaN,NaN,Nordic plant uses from Gunnerus and Høeg
4,3034225,mjølkerot,NaN,NaN,NaN,NaN,NaN,Nordic plant uses from Gunnerus and Høeg
...,...,...,...,...,...,...,...,...
1411683,8633045,pill beetle,en,NaN,NaN,NaN,NaN,The Paleobiology Database
1411684,8491332,bark beetle,en,NaN,NaN,NaN,NaN,The Paleobiology Database
1411685,8567503,crinoid,en,NaN,NaN,NaN,NaN,The Paleobiology Database
1411686,8701826,crinoid,en,NaN,NaN,NaN,NaN,The Paleobiology Database


In [6]:
gbif_vern=gbif_vern[['taxonID','vernacularName','language']]
gbif_vern=gbif_vern[(gbif_vern['language']=='fr')|(gbif_vern['language']=='en')]
# Supprimer les doublons sans tenir compte de la casse et remettre des majuscules en début de chaque mot
gbif_vern = (
    gbif_vern
    .assign(vernacularName=gbif_vern['vernacularName'].str.lower())  # Convertir en minuscules pour ignorer la casse
    .groupby(['taxonID', 'vernacularName', 'language'], as_index=False)  # Grouper par les colonnes d'intérêt
    .size()  # Ajouter une colonne 'size' pour compter les occurrences
    .sort_values(by='size', ascending=False)  # Trier par la fréquence des occurrences
    .drop(columns='size')  # Supprimer la colonne 'size' après avoir trié
    .drop_duplicates(subset=['taxonID', 'vernacularName', 'language'])  # Supprimer les doublons
    .assign(vernacularName=lambda df: df['vernacularName'].str.title())  # Mettre des majuscules en début de chaque mot
)

gbif_vern_grouped = (
    gbif_vern
    .groupby(['taxonID', 'language'], as_index=False)  # Regrouper par taxonID et language sans définir de index
    .first()  # Garder la première ligne de chaque groupe
)

dico_noms_vernaculaires_fr=gbif_vern_grouped[gbif_vern_grouped['language']=='fr']
nouveaux_noms_columns = {'vernacularName': 'vernacularName_fr'}
dico_noms_vernaculaires_fr = dico_noms_vernaculaires_fr.rename(columns=nouveaux_noms_columns)

dico_noms_vernaculaires_en=gbif_vern_grouped[gbif_vern_grouped['language']=='en']
nouveaux_noms_columns = {'vernacularName': 'vernacularName_en'}
dico_noms_vernaculaires_en = dico_noms_vernaculaires_en.rename(columns=nouveaux_noms_columns)

dico_noms_vernaculaires_merged=pd.merge(dico_noms_vernaculaires_fr[['taxonID','vernacularName_fr']],
                                        dico_noms_vernaculaires_en[['taxonID','vernacularName_en']],
                                        on='taxonID',
                                        how="outer")


In [7]:
nouveaux_noms_columns = {'taxonID': 'speciesKey'}
dico_noms_vernaculaires_merged = dico_noms_vernaculaires_merged.rename(columns=nouveaux_noms_columns)
dico_noms_vernaculaires_merged.to_csv(r"C:\Users\anormand\Documents\Projet Python\Biodiv\Data\TAXO_GBIF\dico_noms_vernaculaires_merged.csv", index=False)

In [150]:
nouveaux_noms_columns = {'speciesKey': 'taxonID'}
df_biodiv = df_biodiv.rename(columns=nouveaux_noms_columns)
liste_espece_locale=df_biodiv['taxonID'].unique()
dico_noms_vernaculaires=gbif_vern_grouped[gbif_vern_grouped['taxonID'].isin(liste_espece_locale)]


In [165]:
dico_noms_vernaculaires_merged[(dico_noms_vernaculaires_merged['taxonID']==11909571)]

,taxonID,vernacularName_fr,vernacularName_en
142897,11909571,NaN,Zeledon’S Mouse Opossum


In [ ]:

nouveaux_noms_columns = {'speciesKey': 'taxonID'}
df_biodiv = df_biodiv.rename(columns=nouveaux_noms_columns)

liste_espece_locale=df_biodiv['taxonID'].unique()
dico_taxo_local=dico_taxo[dico_taxo['taxonID'].isin(liste_espece_locale)]